In [ ]:
from oci.ai_vision import AIServiceVisionClient
from oci.ai_vision.models import VideoFeature, TrackingType, VideoObjectDetectionFeature , CreateStreamJobDetails,  CreateStreamSourceDetails, RtspSourceDetails, StreamSourceDetails , StreamGroup,CreateVisionPrivateEndpointDetails, PrivateStreamNetworkAccessDetails , StreamNetworkAccessDetails ,ObjectStorageOutputLocation, UpdateStreamSourceDetails ,UpdateStreamJobDetails , UpdateStreamGroupDetails , UpdateVisionPrivateEndpointDetails , CreateStreamGroupDetails 

import oci, time

In [ ]:
#fill the empty values based as these are required input parameters
CONFIG_PROFILE = ""
COMPARTMENT_ID = ""
STREAM_JOB_DISPLAY_NAME = "stream-job"
SUBNET_ID = ""
CAMERA_URL= ""
NAMESPACE= ""
BUCKET= ""
PREFIX= "testing"
SOURCE_TYPE = "RTSP"
FEATURES = [
    {
        "featureType": "OBJECT_TRACKING",

        "trackingTypes":[
            {
                "objects": ["face"],
                "maxResults": 5,
                "shouldReturnLandmarks": True,
            }
        ]
    }
]

In [ ]:
config = oci.config.from_file('~/.oci/config', profile_name=CONFIG_PROFILE)
endpoint = "https://vision.aiservice.us-ashburn-1.oci.oraclecloud.com"
token_file = config['security_token_file']
with open(token_file, 'r') as f:
   token = f.read()
private_key = oci.signer.load_private_key_from_file(config['key_file'])
signer = oci.auth.signers.SecurityTokenSigner(token, private_key)
ai_service_vision_client = oci.ai_vision.AIServiceVisionClient(config=config, signer=signer, service_endpoint=endpoint)


In [ ]:
# Create Vision Private Endpoint
create_vision_private_endpoint_details = CreateVisionPrivateEndpointDetails()
create_vision_private_endpoint_details.display_name = "pe"
create_vision_private_endpoint_details.subnet_id = SUBNET_ID
create_vision_private_endpoint_details.compartment_id = COMPARTMENT_ID

create_private_endpoint = ai_service_vision_client.create_vision_private_endpoint(create_vision_private_endpoint_details)
while True:
    create_private_endpoint_work_request = ai_service_vision_client.get_work_request(create_private_endpoint.headers['opc-work-request-id'])
    if create_private_endpoint_work_request.data.status  == 'SUCCEEDED':
        private_endpoint_id = create_private_endpoint_work_request.data.id  
        break  
    elif create_private_endpoint_work_request.data.status == 'FAILED' :
        SystemExit("Error Occured while creating Private Endpoint : %s", create_private_endpoint_work_request.message)
        break
    time.sleep(45)


In [ ]:
# Create Stream Source
privateStreamNetworkAccessDetails =PrivateStreamNetworkAccessDetails()
privateStreamNetworkAccessDetails.stream_access_type = "PRIVATE"
privateStreamNetworkAccessDetails.private_endpoint_id = private_endpoint_id


stream_source_deatils = RtspSourceDetails()
stream_source_deatils.camera_url = CAMERA_URL
stream_source_deatils.stream_network_access_details = privateStreamNetworkAccessDetails

create_stream_source_deatils = CreateStreamSourceDetails()
create_stream_source_deatils.compartment_id = COMPARTMENT_ID
create_stream_source_deatils.display_name = "test-source"
create_stream_source_deatils.stream_source_details = stream_source_deatils

create_stream_source_request = ai_service_vision_client.create_stream_source(create_stream_source_deatils)
createworkrequestid = create_stream_source_request.headers['opc-work-request-id']

while True:
    getworkrequestresponse = ai_service_vision_client.get_work_request(createworkrequestid)
    if getworkrequestresponse.data.status == "SUCCEEDED":
        stream_source_id = create_stream_source_request.data.id
        break
    elif getworkrequestresponse.data.status == "FAILED" :
        SystemExit("Error Occured while creating stream source : %s", create_stream_source_request.message)
    time.sleep(45)

In [ ]:
# Create Stream Job
create_stream_job_details = CreateStreamJobDetails()
create_stream_job_details.stream_source_id = stream_source_id

# For Object Tracker (Only "face" supported as of now) - commend if not required
create_stream_job_details.features = FEATURES

output_location = ObjectStorageOutputLocation()
output_location.namespace_name = NAMESPACE
output_location.bucket_name = BUCKET
output_location.prefix = PREFIX
# Choose output location in object storage, make sure you have created the bucket
create_stream_job_details.stream_output_location = output_location

create_stream_job_details.compartment_id = COMPARTMENT_ID
create_stream_job_details.display_name = STREAM_JOB_DISPLAY_NAME


create_stream_job_request = ai_service_vision_client.create_stream_job(create_stream_job_details=create_stream_job_details)

create_stream_job_workrequestid = create_stream_job_request.headers['opc-work-request-id']

while True:
    get_stream_job_workrequest_response = ai_service_vision_client.get_work_request(create_stream_job_workrequestid)
    if get_stream_job_workrequest_response.data.status == "SUCCEEDED":
        stream_job_id = create_stream_job_request.data.id
        break
    elif get_stream_job_workrequest_response.data.status == "FAILED":
        SystemExit("Error Occured while creating stream job : %s", create_stream_job_request.message)
    time.sleep(45)

In [ ]:
# Create Stream Group
streamOverlap = [stream_source_id]
create_stream_group_details = CreateStreamGroupDetails()
create_stream_group_details.compartment_id = COMPARTMENT_ID
create_stream_group_details.display_name = "FunctionalityTest2"
create_stream_group_details.is_enabled = True
create_stream_group_details._stream_source_ids = streamOverlap

create_stream_group_request = ai_service_vision_client.create_stream_group(create_stream_group_details)

while True:
    create_streamgroup__work_request = ai_service_vision_client.get_work_request(create_stream_group_request.headers['opc-work-request-id'])
    if getworkrequestresponse.data.status == "SUCCEEDED":
        stream_group_id = create_stream_group_request.data.id
        break
    elif create_streamgroup__work_request.data.status == "FAILED" :
        SystemExit("Error Occured while creating stream group : %s", create_stream_group_request.message)
    time.sleep(20)

In [ ]:
# Start Stream Job
start_stream_job_request = ai_service_vision_client.start_stream_job(stream_job_id)

while True:
    start_stream_job_work_request = ai_service_vision_client.get_work_request(start_stream_job_request.headers['opc-work-request-id'])
    if start_stream_job_work_request.data.status  == 'SUCCEEDED':
        ##success 
        break
    elif start_stream_job_work_request.data.status == 'FAILED' :
        SystemExit("Error Occured while starting stream job : %s", start_stream_job_request.message)
    time.sleep(45)

In [ ]:
# Stop Stream Job
stop_stream_job_request = ai_service_vision_client.stop_stream_job(stream_job_id)

while True:
    stop_stream_job_work_request = ai_service_vision_client.get_work_request(stop_stream_job_request.headers['opc-work-request-id'])
    if stop_stream_job_work_request.data.status  == 'SUCCEEDED':
        break
    elif stop_stream_job_work_request.data.status == 'FAILED':
        SystemExit("Error occured while stoping stream job : %s", stop_stream_job_request.message)
    time.sleep(45)


In [ ]:
# delete_stream_group
stream_group_delete_request = ai_service_vision_client.delete_stream_group(stream_group_id)

while True:
    stream_group_delete_work_request = ai_service_vision_client.get_work_request(stream_group_delete_request.headers['opc-work-request-id'])
    if stream_group_delete_work_request.data.status  == 'SUCCEEDED' :
        break
    elif stream_group_delete_work_request.data.status == 'FAILED' :
        SystemExit("Error occured while deleting stream group : %s", stream_group_delete_request.message)
    time.sleep(20)

In [ ]:
# delete_stream_job
stream_job_delete_request = ai_service_vision_client.delete_stream_job(stream_job_id)

while True:
    stream_job_delete_work_request = ai_service_vision_client.get_work_request(stream_job_delete_request.headers['opc-work-request-id'])
    if stream_job_delete_work_request.data.status  == 'SUCCEEDED':
        break
    elif stream_job_delete_work_request.data.status == 'FAILED' :
        SystemExit("Error occured while deleting stream job : %s", stream_job_delete_request.message)
    time.sleep(30)

In [ ]:
# delete_stream_source
delete_stream_source_request = ai_service_vision_client.delete_stream_source(stream_source_id)

while True:
    delete_stream_source_work_request = ai_service_vision_client.get_work_request(delete_stream_source_request.headers['opc-work-request-id'])
    print(delete_stream_source_work_request.data.status)
    if delete_stream_source_work_request.data.status  == 'SUCCEEDED':
        break
    elif delete_stream_source_work_request.data.status == 'FAILED' :
        SystemExit("Error occured while deleting stream source : %s", delete_stream_source_request.message)
    time.sleep(20)

In [ ]:
# delete vision private endpoint
delete_private_endpoint_request = ai_service_vision_client.delete_vision_private_endpoint(private_endpoint_id)

while True:
    delete_private_endpoint_work_request = ai_service_vision_client.get_work_request(delete_private_endpoint_request.headers['opc-work-request-id'])
    if delete_private_endpoint_work_request.data.status  == 'SUCCEEDED' :
        break
    elif delete_private_endpoint_work_request.data.status == 'FAILED' :
        SystemExit("Error occured while deleting private endpoint : %s", delete_private_endpoint_request.message)

    time.sleep(120)